<a href="https://colab.research.google.com/github/dougscohen/DS-Unit-2-Linear-Models/blob/master/module3-ridge-regression/LS_DS_213_assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Lambda School Data Science

*Unit 2, Sprint 1, Module 3*

---

# Ridge Regression

## Assignment

We're going back to our other **New York City** real estate dataset. Instead of predicting apartment rents, you'll predict property sales prices.

But not just for condos in Tribeca...

- [ ] Use a subset of the data where `BUILDING_CLASS_CATEGORY` == `'01 ONE FAMILY DWELLINGS'` and the sale price was more than 100 thousand and less than 2 million.
- [ ] Do train/test split. Use data from January — March 2019 to train. Use data from April 2019 to test.
- [ ] Do one-hot encoding of categorical features.
- [ ] Do feature selection with `SelectKBest`.
- [ ] Fit a ridge regression model with multiple features. Use the `normalize=True` parameter (or do [feature scaling](https://scikit-learn.org/stable/modules/preprocessing.html) beforehand — use the scaler's `fit_transform` method with the train set, and the scaler's `transform` method with the test set)
- [ ] Get mean absolute error for the test set.
- [ ] As always, commit your notebook to your fork of the GitHub repo.

The [NYC Department of Finance](https://www1.nyc.gov/site/finance/taxes/property-rolling-sales-data.page) has a glossary of property sales terms and NYC Building Class Code Descriptions. The data comes from the [NYC OpenData](https://data.cityofnewyork.us/browse?q=NYC%20calendar%20sales) portal.


## Stretch Goals

Don't worry, you aren't expected to do all these stretch goals! These are just ideas to consider and choose from.

- [ ] Add your own stretch goal(s) !
- [ ] Instead of `Ridge`, try `LinearRegression`. Depending on how many features you select, your errors will probably blow up! 💥
- [ ] Instead of `Ridge`, try [`RidgeCV`](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.RidgeCV.html).
- [ ] Learn more about feature selection:
    - ["Permutation importance"](https://www.kaggle.com/dansbecker/permutation-importance)
    - [scikit-learn's User Guide for Feature Selection](https://scikit-learn.org/stable/modules/feature_selection.html)
    - [mlxtend](http://rasbt.github.io/mlxtend/) library
    - scikit-learn-contrib libraries: [boruta_py](https://github.com/scikit-learn-contrib/boruta_py) & [stability-selection](https://github.com/scikit-learn-contrib/stability-selection)
    - [_Feature Engineering and Selection_](http://www.feat.engineering/) by Kuhn & Johnson.
- [ ] Try [statsmodels](https://www.statsmodels.org/stable/index.html) if you’re interested in more inferential statistical approach to linear regression and feature selection, looking at p values and 95% confidence intervals for the coefficients.
- [ ] Read [_An Introduction to Statistical Learning_](http://faculty.marshall.usc.edu/gareth-james/ISL/ISLR%20Seventh%20Printing.pdf), Chapters 1-3, for more math & theory, but in an accessible, readable way.
- [ ] Try [scikit-learn pipelines](https://scikit-learn.org/stable/modules/compose.html).

In [0]:
%%capture
import sys

# If you're on Colab:
if 'google.colab' in sys.modules:
    DATA_PATH = 'https://raw.githubusercontent.com/LambdaSchool/DS-Unit-2-Applied-Modeling/master/data/'
    !pip install category_encoders==2.*

# If you're working locally:
else:
    DATA_PATH = '../data/'
    
# Ignore this Numpy warning when using Plotly Express:
# FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
import warnings
warnings.filterwarnings(action='ignore', category=FutureWarning, module='numpy')

In [0]:
import pandas as pd
import pandas_profiling

# Read New York City property sales data
df = pd.read_csv(DATA_PATH+'condos/NYC_Citywide_Rolling_Calendar_Sales.csv')

# Change column names: replace spaces with underscores
df.columns = [col.replace(' ', '_') for col in df]

# SALE_PRICE was read as strings.
# Remove symbols, convert to integer
df['SALE_PRICE'] = (
    df['SALE_PRICE']
    .str.replace('$','')
    .str.replace('-','')
    .str.replace(',','')
    .astype(int)
)

In [0]:
# BOROUGH is a numeric column, but arguably should be a categorical feature,
# so convert it from a number to a string
df['BOROUGH'] = df['BOROUGH'].astype(str)

In [0]:
# Reduce cardinality for NEIGHBORHOOD feature

# Get a list of the top 10 neighborhoods
top10 = df['NEIGHBORHOOD'].value_counts()[:10].index

# At locations where the neighborhood is NOT in the top 10, 
# replace the neighborhood with 'OTHER'
df.loc[~df['NEIGHBORHOOD'].isin(top10), 'NEIGHBORHOOD'] = 'OTHER'

In [216]:
# See initial shape and head of datframe

print(df.shape)
df.head()

(23040, 21)


,BOROUGH,NEIGHBORHOOD,BUILDING_CLASS_CATEGORY,TAX_CLASS_AT_PRESENT,BLOCK,LOT,EASE-MENT,BUILDING_CLASS_AT_PRESENT,ADDRESS,APARTMENT_NUMBER,ZIP_CODE,RESIDENTIAL_UNITS,COMMERCIAL_UNITS,TOTAL_UNITS,LAND_SQUARE_FEET,GROSS_SQUARE_FEET,YEAR_BUILT,TAX_CLASS_AT_TIME_OF_SALE,BUILDING_CLASS_AT_TIME_OF_SALE,SALE_PRICE,SALE_DATE
0,1,OTHER,13 CONDOS - ELEVATOR APARTMENTS,2,716,1246,NaN,R4,"447 WEST 18TH STREET, PH12A",PH12A,10011.0,1.0,0.0,1.0,"10,733",1979.0,2007.0,2,R4,0,01/01/2019
1,1,OTHER,21 OFFICE BUILDINGS,4,812,68,NaN,O5,144 WEST 37TH STREET,NaN,10018.0,0.0,6.0,6.0,"2,962",15435.0,1920.0,4,O5,0,01/01/2019
2,1,OTHER,21 OFFICE BUILDINGS,4,839,69,NaN,O5,40 WEST 38TH STREET,NaN,10018.0,0.0,7.0,7.0,"2,074",11332.0,1930.0,4,O5,0,01/01/2019
3,1,OTHER,13 CONDOS - ELEVATOR APARTMENTS,2,592,1041,NaN,R4,"1 SHERIDAN SQUARE, 8C",8C,10014.0,1.0,0.0,1.0,0,500.0,0.0,2,R4,0,01/01/2019
4,1,UPPER EAST SIDE (59-79),15 CONDOS - 2-10 UNIT RESIDENTIAL,2C,1379,1402,NaN,R1,"20 EAST 65TH STREET, B",B,10065.0,1.0,0.0,1.0,0,6406.0,0.0,2,R1,0,01/01/2019


In [217]:
# Check out the null values to see if we need to drop any columns

df.isnull().sum()

# 'EASE-MENT' column is entirely null values
# 'APARTMENT_NUMBER is 77% null values

BOROUGH                               0
NEIGHBORHOOD                          0
BUILDING_CLASS_CATEGORY               0
TAX_CLASS_AT_PRESENT                  1
BLOCK                                 0
LOT                                   0
EASE-MENT                         23040
BUILDING_CLASS_AT_PRESENT             1
ADDRESS                               0
APARTMENT_NUMBER                  17839
ZIP_CODE                              1
RESIDENTIAL_UNITS                     1
COMMERCIAL_UNITS                      1
TOTAL_UNITS                           1
LAND_SQUARE_FEET                     53
GROSS_SQUARE_FEET                     1
YEAR_BUILT                           35
TAX_CLASS_AT_TIME_OF_SALE             0
BUILDING_CLASS_AT_TIME_OF_SALE        0
SALE_PRICE                            0
SALE_DATE                             0
dtype: int64

In [218]:
# Override the current dataframe with conditions based on the information 
# above in the directions

# Only want Building Class Category to be equal to '01 ONE FAMILY DWELLINGS'
# WANT Sale Price to be above 100,000 and also below 2,000,000

df = df[(df['BUILDING_CLASS_CATEGORY'] == '01 ONE FAMILY DWELLINGS') &
               (df['SALE_PRICE'] > 100000) &
               (df['SALE_PRICE'] < 2000000)]

# See the new shape after conditioning

print(df.shape)
df.head()

(3151, 21)


,BOROUGH,NEIGHBORHOOD,BUILDING_CLASS_CATEGORY,TAX_CLASS_AT_PRESENT,BLOCK,LOT,EASE-MENT,BUILDING_CLASS_AT_PRESENT,ADDRESS,APARTMENT_NUMBER,ZIP_CODE,RESIDENTIAL_UNITS,COMMERCIAL_UNITS,TOTAL_UNITS,LAND_SQUARE_FEET,GROSS_SQUARE_FEET,YEAR_BUILT,TAX_CLASS_AT_TIME_OF_SALE,BUILDING_CLASS_AT_TIME_OF_SALE,SALE_PRICE,SALE_DATE
44,3,OTHER,01 ONE FAMILY DWELLINGS,1,5495,801,NaN,A9,4832 BAY PARKWAY,NaN,11230.0,1.0,0.0,1.0,"6,800",1325.0,1930.0,1,A9,550000,01/01/2019
61,4,OTHER,01 ONE FAMILY DWELLINGS,1,7918,72,NaN,A1,80-23 232ND STREET,NaN,11427.0,1.0,0.0,1.0,"4,000",2001.0,1940.0,1,A1,200000,01/01/2019
78,2,OTHER,01 ONE FAMILY DWELLINGS,1,4210,19,NaN,A1,1260 RHINELANDER AVE,NaN,10461.0,1.0,0.0,1.0,"3,500",2043.0,1925.0,1,A1,810000,01/02/2019
108,3,OTHER,01 ONE FAMILY DWELLINGS,1,5212,69,NaN,A1,469 E 25TH ST,NaN,11226.0,1.0,0.0,1.0,"4,000",2680.0,1899.0,1,A1,125000,01/02/2019
111,3,OTHER,01 ONE FAMILY DWELLINGS,1,7930,121,NaN,A5,5521 WHITTY LANE,NaN,11203.0,1.0,0.0,1.0,"1,710",1872.0,1940.0,1,A5,620000,01/02/2019


In [219]:
# Check out the datatype of the 'SALE_DATE' column

df.dtypes.tail()

YEAR_BUILT                        float64
TAX_CLASS_AT_TIME_OF_SALE           int64
BUILDING_CLASS_AT_TIME_OF_SALE     object
SALE_PRICE                          int64
SALE_DATE                          object
dtype: object

In [0]:
# Change Sale Date datatype to datetime 

df['SALE_DATE'] = pd.to_datetime(df['SALE_DATE'], infer_datetime_format=True)

In [221]:
# Put out the frist row to see what the format looks like

df['SALE_DATE'].iloc[0]

Timestamp('2019-01-01 00:00:00')

In [222]:
# Establish a cutoff of where we want to split the dataset and assign it to a 
# variable

cutoff = pd.to_datetime('2019-04-01')

# Split the dataset into training and testing data based on time (April and May)
train = df[df['SALE_DATE'] < cutoff]
test = df[df['SALE_DATE'] >= cutoff]

# See the shapes of each new dataframe

train.shape, test.shape

((2507, 21), (644, 21))

In [223]:
# Descriptive statistics for each numeric column

train.describe(include='number')

,BLOCK,LOT,EASE-MENT,ZIP_CODE,RESIDENTIAL_UNITS,COMMERCIAL_UNITS,TOTAL_UNITS,GROSS_SQUARE_FEET,YEAR_BUILT,TAX_CLASS_AT_TIME_OF_SALE,SALE_PRICE
count,2507.000000,2507.000000,0.0,2507.000000,2507.000000,2507.000000,2507.000000,2507.000000,2507.000000,2507.0,2.507000e+03
mean,6758.303949,75.778221,NaN,10993.398484,0.987635,0.016354,1.003989,1473.744715,1944.766653,1.0,6.215737e+05
std,3975.909029,157.531138,NaN,494.291462,0.110532,0.129966,0.171794,599.217635,27.059337,0.0,2.916072e+05
min,21.000000,1.000000,NaN,10301.000000,0.000000,0.000000,0.000000,0.000000,1890.000000,1.0,1.040000e+05
25%,3837.500000,21.000000,NaN,10314.000000,1.000000,0.000000,1.000000,1144.000000,1925.000000,1.0,4.405000e+05
50%,6022.000000,42.000000,NaN,11234.000000,1.000000,0.000000,1.000000,1368.000000,1940.000000,1.0,5.600000e+05
75%,9888.500000,70.000000,NaN,11413.000000,1.000000,0.000000,1.000000,1683.000000,1960.000000,1.0,7.500000e+05
max,16350.000000,2720.000000,NaN,11697.000000,1.000000,2.000000,3.000000,7875.000000,2018.000000,1.0,1.955000e+06


In [224]:
# Descriptive statistics for each categorical column
# Transposed the dataframe and then sorted by # of unique values so I can
# easy see which columns have high cardinality 

train.describe(exclude='number').T.sort_values(by='unique')

,count,unique,top,freq,first,last
BUILDING_CLASS_CATEGORY,2507,1,01 ONE FAMILY DWELLINGS,2507,NaT,NaT
APARTMENT_NUMBER,1,1,RP.,1,NaT,NaT
TAX_CLASS_AT_PRESENT,2507,2,1,2476,NaT,NaT
BOROUGH,2507,5,4,1204,NaT,NaT
NEIGHBORHOOD,2507,7,OTHER,2360,NaT,NaT
BUILDING_CLASS_AT_TIME_OF_SALE,2507,11,A1,919,NaT,NaT
BUILDING_CLASS_AT_PRESENT,2507,13,A1,919,NaT,NaT
SALE_DATE,2507,68,2019-01-31 00:00:00,78,2019-01-01,2019-03-30
LAND_SQUARE_FEET,2507,887,"4,000",234,NaT,NaT
ADDRESS,2507,2497,118-20 202ND STREET,2,NaT,NaT


In [0]:
# Set the target or dependent variable

target = 'SALE_PRICE'

# Make a list of high cardinality columns and also null columns that we want to
# drop

high_cardinality = ['SALE_DATE', 'LAND_SQUARE_FEET', 'ADDRESS']
null_columns = ['EASE-MENT', 'APARTMENT_NUMBER']

# Set the feature or dependent variable(s) by dropping which columns we don't
# want to use
features = train.columns.drop([target] + high_cardinality + null_columns)

In [0]:
# 3. Arrange X features matrix and y target vector

X_train = train[features]
y_train = train[target]
X_test = test[features]
y_test = test[target]

In [227]:
# X_train before one hot encoding (shows 15 columns)

print(X_train.shape)
X_train.head()

(2507, 15)


,BOROUGH,NEIGHBORHOOD,BUILDING_CLASS_CATEGORY,TAX_CLASS_AT_PRESENT,BLOCK,LOT,BUILDING_CLASS_AT_PRESENT,ZIP_CODE,RESIDENTIAL_UNITS,COMMERCIAL_UNITS,TOTAL_UNITS,GROSS_SQUARE_FEET,YEAR_BUILT,TAX_CLASS_AT_TIME_OF_SALE,BUILDING_CLASS_AT_TIME_OF_SALE
44,3,OTHER,01 ONE FAMILY DWELLINGS,1,5495,801,A9,11230.0,1.0,0.0,1.0,1325.0,1930.0,1,A9
61,4,OTHER,01 ONE FAMILY DWELLINGS,1,7918,72,A1,11427.0,1.0,0.0,1.0,2001.0,1940.0,1,A1
78,2,OTHER,01 ONE FAMILY DWELLINGS,1,4210,19,A1,10461.0,1.0,0.0,1.0,2043.0,1925.0,1,A1
108,3,OTHER,01 ONE FAMILY DWELLINGS,1,5212,69,A1,11226.0,1.0,0.0,1.0,2680.0,1899.0,1,A1
111,3,OTHER,01 ONE FAMILY DWELLINGS,1,7930,121,A5,11203.0,1.0,0.0,1.0,1872.0,1940.0,1,A5


In [0]:
# import category encoders
import category_encoders as ce

# Instantiate the class
encoder = ce.OneHotEncoder(use_cat_names=True)

# Fit the class to the training data
X_train = encoder.fit_transform(X_train)

# Transform the testing data
X_test = encoder.transform(X_test)

In [229]:
# X_train after one hot encoding (shows 48 columns)

print(X_train.shape)
X_train.head()

(2507, 48)


,BOROUGH_3,BOROUGH_4,BOROUGH_2,BOROUGH_5,BOROUGH_1,NEIGHBORHOOD_OTHER,NEIGHBORHOOD_FLUSHING-NORTH,NEIGHBORHOOD_EAST NEW YORK,NEIGHBORHOOD_BEDFORD STUYVESANT,NEIGHBORHOOD_FOREST HILLS,NEIGHBORHOOD_BOROUGH PARK,NEIGHBORHOOD_ASTORIA,BUILDING_CLASS_CATEGORY_01 ONE FAMILY DWELLINGS,TAX_CLASS_AT_PRESENT_1,TAX_CLASS_AT_PRESENT_1D,BLOCK,LOT,BUILDING_CLASS_AT_PRESENT_A9,BUILDING_CLASS_AT_PRESENT_A1,BUILDING_CLASS_AT_PRESENT_A5,BUILDING_CLASS_AT_PRESENT_A0,BUILDING_CLASS_AT_PRESENT_A2,BUILDING_CLASS_AT_PRESENT_A3,BUILDING_CLASS_AT_PRESENT_S1,BUILDING_CLASS_AT_PRESENT_A4,BUILDING_CLASS_AT_PRESENT_A6,BUILDING_CLASS_AT_PRESENT_A8,BUILDING_CLASS_AT_PRESENT_B2,BUILDING_CLASS_AT_PRESENT_S0,BUILDING_CLASS_AT_PRESENT_B3,ZIP_CODE,RESIDENTIAL_UNITS,COMMERCIAL_UNITS,TOTAL_UNITS,GROSS_SQUARE_FEET,YEAR_BUILT,TAX_CLASS_AT_TIME_OF_SALE,BUILDING_CLASS_AT_TIME_OF_SALE_A9,BUILDING_CLASS_AT_TIME_OF_SALE_A1,BUILDING_CLASS_AT_TIME_OF_SALE_A5,BUILDING_CLASS_AT_TIME_OF_SALE_A0,BUILDING_CLASS_AT_TIME_OF_SALE_A2,BUILDING_CLASS_AT_TIME_OF_SALE_A3,BUILDING_CLASS_AT_TIME_OF_SALE_S1,BUILDING_CLASS_AT_TIME_OF_SALE_A4,BUILDING_CLASS_AT_TIME_OF_SALE_A6,BUILDING_CLASS_AT_TIME_OF_SALE_A8,BUILDING_CLASS_AT_TIME_OF_SALE_S0
44,1,0,0,0,0,1,0,0,0,0,0,0,1,1,0,5495,801,1,0,0,0,0,0,0,0,0,0,0,0,0,11230.0,1.0,0.0,1.0,1325.0,1930.0,1,1,0,0,0,0,0,0,0,0,0,0
61,0,1,0,0,0,1,0,0,0,0,0,0,1,1,0,7918,72,0,1,0,0,0,0,0,0,0,0,0,0,0,11427.0,1.0,0.0,1.0,2001.0,1940.0,1,0,1,0,0,0,0,0,0,0,0,0
78,0,0,1,0,0,1,0,0,0,0,0,0,1,1,0,4210,19,0,1,0,0,0,0,0,0,0,0,0,0,0,10461.0,1.0,0.0,1.0,2043.0,1925.0,1,0,1,0,0,0,0,0,0,0,0,0
108,1,0,0,0,0,1,0,0,0,0,0,0,1,1,0,5212,69,0,1,0,0,0,0,0,0,0,0,0,0,0,11226.0,1.0,0.0,1.0,2680.0,1899.0,1,0,1,0,0,0,0,0,0,0,0,0
111,1,0,0,0,0,1,0,0,0,0,0,0,1,1,0,7930,121,0,0,1,0,0,0,0,0,0,0,0,0,0,11203.0,1.0,0.0,1.0,1872.0,1940.0,1,0,0,1,0,0,0,0,0,0,0,0


In [230]:
from sklearn.feature_selection import SelectKBest, f_regression

# Select k number of features that best correlate with the target 
# (here I chose 15)

selector = SelectKBest(score_func=f_regression, k=15)

# .fit_transform on the train set
# .transform on test set

X_train_selected = selector.fit_transform(X_train, y_train)
X_test_selected = selector.transform(X_test)

print(X_train_selected.shape)
X_train_selected

(2507, 15)


/usr/local/lib/python3.6/dist-packages/sklearn/feature_selection/_univariate_selection.py:299: RuntimeWarning: divide by zero encountered in true_divide
  corr /= X_norms
/usr/local/lib/python3.6/dist-packages/sklearn/feature_selection/_univariate_selection.py:304: RuntimeWarning: invalid value encountered in true_divide
  F = corr ** 2 / (1 - corr ** 2) * degrees_of_freedom
/usr/local/lib/python3.6/dist-packages/scipy/stats/_distn_infrastructure.py:903: RuntimeWarning: invalid value encountered in greater
  return (a < x) & (x < b)
/usr/local/lib/python3.6/dist-packages/scipy/stats/_distn_infrastructure.py:903: RuntimeWarning: invalid value encountered in less
  return (a < x) & (x < b)
/usr/local/lib/python3.6/dist-packages/scipy/stats/_distn_infrastructure.py:1912: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= _a)


array([[1.000e+00, 0.000e+00, 0.000e+00, ..., 1.325e+03, 0.000e+00,
        0.000e+00],
       [0.000e+00, 0.000e+00, 0.000e+00, ..., 2.001e+03, 0.000e+00,
        0.000e+00],
       [0.000e+00, 1.000e+00, 0.000e+00, ..., 2.043e+03, 0.000e+00,
        0.000e+00],
       ...,
       [0.000e+00, 0.000e+00, 1.000e+00, ..., 1.807e+03, 0.000e+00,
        0.000e+00],
       [0.000e+00, 0.000e+00, 1.000e+00, ..., 6.210e+02, 0.000e+00,
        0.000e+00],
       [0.000e+00, 0.000e+00, 0.000e+00, ..., 1.163e+03, 0.000e+00,
        0.000e+00]])

In [231]:
# figure out which features were selected

selected_mask = selector.get_support()
all_names = X_train.columns
selected_names = all_names[selected_mask]
unselected_names = all_names[~selected_mask]

print('Features selected:')
for name in selected_names:
  print(name)


print('\n')
print('Features not selected:')
for name in unselected_names:
  print(name)

Features selected:
BOROUGH_3
BOROUGH_2
BOROUGH_5
NEIGHBORHOOD_OTHER
NEIGHBORHOOD_FLUSHING-NORTH
NEIGHBORHOOD_FOREST HILLS
BLOCK
BUILDING_CLASS_AT_PRESENT_A5
BUILDING_CLASS_AT_PRESENT_A3
ZIP_CODE
COMMERCIAL_UNITS
TOTAL_UNITS
GROSS_SQUARE_FEET
BUILDING_CLASS_AT_TIME_OF_SALE_A5
BUILDING_CLASS_AT_TIME_OF_SALE_A3


Features not selected:
BOROUGH_4
BOROUGH_1
NEIGHBORHOOD_EAST NEW YORK
NEIGHBORHOOD_BEDFORD STUYVESANT
NEIGHBORHOOD_BOROUGH PARK
NEIGHBORHOOD_ASTORIA
BUILDING_CLASS_CATEGORY_01 ONE FAMILY DWELLINGS
TAX_CLASS_AT_PRESENT_1
TAX_CLASS_AT_PRESENT_1D
LOT
BUILDING_CLASS_AT_PRESENT_A9
BUILDING_CLASS_AT_PRESENT_A1
BUILDING_CLASS_AT_PRESENT_A0
BUILDING_CLASS_AT_PRESENT_A2
BUILDING_CLASS_AT_PRESENT_S1
BUILDING_CLASS_AT_PRESENT_A4
BUILDING_CLASS_AT_PRESENT_A6
BUILDING_CLASS_AT_PRESENT_A8
BUILDING_CLASS_AT_PRESENT_B2
BUILDING_CLASS_AT_PRESENT_S0
BUILDING_CLASS_AT_PRESENT_B3
RESIDENTIAL_UNITS
YEAR_BUILT
TAX_CLASS_AT_TIME_OF_SALE
BUILDING_CLASS_AT_TIME_OF_SALE_A9
BUILDING_CLASS_AT_TIME_OF_SALE_A

In [232]:
# import Ridge Estimator

from sklearn.linear_model import Ridge

# Instantiate the class
ridge = Ridge(normalize=True)

# Fit the ridge model
ridge.fit(X_train, y_train)

# Apply the model to our testing data
y_pred = ridge.predict(X_test)

# Print the coefficients of 48 features, and the y-intercept
print(ridge.coef_)
print(ridge.intercept_)

[ 7.79420956e+04  1.60069256e+04 -6.91466978e+04 -4.32386065e+04
  7.08427553e+04 -5.39267218e+04  7.87518807e+04 -1.64302287e+05
 -1.23191727e+05  1.95152823e+05  1.22115926e+05  1.27523278e+05
  0.00000000e+00  2.07945724e+03 -2.07945724e+03 -7.83716891e+00
 -3.42335074e+01 -6.79704129e+03  4.52730802e+03 -1.80697944e+04
  2.40844063e+04  2.85199072e+03  1.30279815e+05  1.95395966e+04
  5.59977841e+04 -8.49459518e+04 -2.07945724e+03  5.29472571e+04
  3.47832261e+05  2.23053594e+05  4.34445576e+01  2.07945724e+03
  3.41505893e+04  2.04061707e+04  9.24171602e+01  1.69178297e+02
  0.00000000e+00 -6.79704129e+03  4.52730802e+03 -1.80028325e+04
  2.40844063e+04  3.53643222e+03  1.30279815e+05  1.95395966e+04
  5.59977841e+04 -8.49459518e+04 -2.07945724e+03  3.47832261e+05]
-243019.37959392508


In [233]:
y_pred

array([ 582522.56747321,  468754.7738621 ,  783913.21485276,
        658899.06144818,  608417.17153152,  685264.72595552,
        659807.00301365,  711295.53009624,  708813.3828299 ,
        732657.48538108,  678527.15855994,  419089.9680088 ,
        407029.86873332,  723017.12513096,  553493.14075098,
        613276.97720118,  630448.94815481,  937004.22027324,
        590032.74286186,  621328.88336165,  861886.96280727,
        560964.53336557,  651124.07399361,  623314.83223903,
        656493.26205372,  544042.98584778,  507912.18999652,
        737447.99257667,  591137.83735618,  606456.38796216,
        572532.31495481,  579016.12721551,  559462.23977341,
        509340.16074189,  820060.22884484,  637436.74911422,
        600813.12479697,  593205.35133113,  516006.28430117,
        524129.17893144,  462313.57938036,  496074.39974214,
        899944.95100948,  698385.86626182,  676164.14882539,
        659732.0770719 ,  817418.51680429,  685190.29312286,
        631800.56789567,

In [234]:
y_test

18235     895000
18239     253500
18244    1300000
18280     789000
18285     525000
          ...   
23029     635000
23031     514000
23032     635000
23033     545000
23035     510000
Name: SALE_PRICE, Length: 644, dtype: int64

In [235]:
from sklearn.metrics import mean_absolute_error

# Calculate the mean absolute error of the testing dataset

mae = mean_absolute_error(y_test, y_pred)
print(f'Test Mean Absolute Error after Ridge Regression: ${mae:.0f}')

Test Mean Absolute Error after Ridge Regression: $172238


In [236]:
# Run Ridge Regression with Cross Validation
# Here I import the class, substantiate it, fit it the model, and then apply 
# the model

from sklearn.linear_model import RidgeCV

# list of alpha values to use

alphas = [.001, .01, .1, 1, 10, 100, 1000]

ridgeCV = RidgeCV(alphas=alphas, normalize=True)
ridgeCV.fit(X_train, y_train)
y_pred = ridgeCV.predict(X_test)
ridgeCV.alpha_


0.001

In [237]:
from sklearn.metrics import mean_absolute_error

# Testing MAE after doing Ridge Regression with Cross Validation

mae = mean_absolute_error(y_test, y_pred)
print(f'Test Mean Absolute Error from doing Ridge Regression with Cross Validation: ${mae:.0f}')

Test Mean Absolute Error from doing Ridge Regression with Cross Validation: $158360


In [0]:
# Run Ridge Regression on the 15 columns we got from Select K Best
# Here I import the class, substantiate it, fit it the model, and then apply 
# the model

from sklearn.linear_model import Ridge

ridge = Ridge(normalize=True)
ridge.fit(X_train[selected_names], y_train)
y_pred = ridge.predict(X_test[selected_names])

In [239]:
# Calculate the mean absolute error of the testing dataset after fitting the
# model to the more specific training set above

mae = mean_absolute_error(y_test, y_pred)
print(f'Test Mean Absolute Error from doing Ridge Regression on 15 K Best columns: ${mae:.0f}')

Test Mean Absolute Error from doing Ridge Regression on 15 K Best columns: $176240


In [240]:
# Run Ridge Regression with Cross Validation on the 15 Select K Best columns
# Here I import the class, substantiate it, fit it the model, and then apply 
# the model


from sklearn.linear_model import RidgeCV

# list of alphas to use

alphas = [.001, .01, .1, 1, 10, 100, 1000]

ridgeCV = RidgeCV(alphas=alphas, normalize=True)
ridgeCV.fit(X_train[selected_names], y_train)
y_pred = ridgeCV.predict(X_test[selected_names])
ridgeCV.alpha_


0.001

In [241]:
# Testing MAE after doing Ridge Regression with Cross Validation on 15
# Select K Best columns

mae = mean_absolute_error(y_test, y_pred)
print(f'Test Mean Absolute Error from doing Ridge Regression with Cross \nValidation on 15 K Best columns: ${mae:.0f}')

Test Mean Absolute Error from doing Ridge Regression with Cross 
Validation on 15 K Best columns: $162429


In [242]:
# TODO: How many features should be selected?

# You can try a range of values for k,
# then choose the model with the best score.
# If multiple models "tie" for the best score,
# choose the simplest model.
# You decide what counts as a tie!

from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error

for k in range(1, len(X_train.columns)+1):
    print(f'{k} features')
    
    selector = SelectKBest(score_func=f_regression, k=k)
    X_train_selected = selector.fit_transform(X_train, y_train)
    X_test_selected = selector.transform(X_test)

    model = LinearRegression()
    model.fit(X_train_selected, y_train)
    y_pred = model.predict(X_test_selected)
    mae = mean_absolute_error(y_test, y_pred)
    print(f'Test Mean Absolute Error: ${mae:,.0f} \n')

1 features
Test Mean Absolute Error: $183,641 

2 features
Test Mean Absolute Error: $182,569 

3 features
Test Mean Absolute Error: $182,569 

4 features
Test Mean Absolute Error: $183,441 

5 features
Test Mean Absolute Error: $181,935 

6 features
Test Mean Absolute Error: $176,456 

7 features
Test Mean Absolute Error: $177,001 

8 features
Test Mean Absolute Error: $176,707 

9 features
Test Mean Absolute Error: $171,663 

10 features
Test Mean Absolute Error: $170,977 

11 features
Test Mean Absolute Error: $170,507 

12 features
Test Mean Absolute Error: $162,301 

13 features
Test Mean Absolute Error: $163,559 

14 features
Test Mean Absolute Error: $162,562 

15 features
Test Mean Absolute Error: $162,564 

16 features
Test Mean Absolute Error: $162,419 

17 features
Test Mean Absolute Error: $162,419 

18 features
Test Mean Absolute Error: $162,177 

19 features
Test Mean Absolute Error: $162,177 

20 features
Test Mean Absolute Error: $157,893 

21 features


/usr/local/lib/python3.6/dist-packages/sklearn/feature_selection/_univariate_selection.py:299: RuntimeWarning: divide by zero encountered in true_divide
  corr /= X_norms
/usr/local/lib/python3.6/dist-packages/sklearn/feature_selection/_univariate_selection.py:304: RuntimeWarning: invalid value encountered in true_divide
  F = corr ** 2 / (1 - corr ** 2) * degrees_of_freedom
/usr/local/lib/python3.6/dist-packages/scipy/stats/_distn_infrastructure.py:903: RuntimeWarning: invalid value encountered in greater
  return (a < x) & (x < b)
/usr/local/lib/python3.6/dist-packages/scipy/stats/_distn_infrastructure.py:903: RuntimeWarning: invalid value encountered in less
  return (a < x) & (x < b)
/usr/local/lib/python3.6/dist-packages/scipy/stats/_distn_infrastructure.py:1912: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= _a)
/usr/local/lib/python3.6/dist-packages/sklearn/feature_selection/_univariate_selection.py:299: RuntimeWarning: divide by zero encountered

Test Mean Absolute Error: $157,966 

22 features
Test Mean Absolute Error: $157,966 

23 features
Test Mean Absolute Error: $157,966 

24 features
Test Mean Absolute Error: $157,966 

25 features
Test Mean Absolute Error: $157,697 

26 features
Test Mean Absolute Error: $157,578 

27 features
Test Mean Absolute Error: $157,511 

28 features
Test Mean Absolute Error: $157,468 

29 features
Test Mean Absolute Error: $158,331 

30 features
Test Mean Absolute Error: $158,428 

31 features
Test Mean Absolute Error: $158,428 

32 features
Test Mean Absolute Error: $158,429 

33 features
Test Mean Absolute Error: $158,084 

34 features
Test Mean Absolute Error: $158,062 

35 features
Test Mean Absolute Error: $158,086 

36 features
Test Mean Absolute Error: $157,741 

37 features


/usr/local/lib/python3.6/dist-packages/scipy/stats/_distn_infrastructure.py:903: RuntimeWarning: invalid value encountered in greater
  return (a < x) & (x < b)
/usr/local/lib/python3.6/dist-packages/scipy/stats/_distn_infrastructure.py:903: RuntimeWarning: invalid value encountered in less
  return (a < x) & (x < b)
/usr/local/lib/python3.6/dist-packages/scipy/stats/_distn_infrastructure.py:1912: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= _a)
/usr/local/lib/python3.6/dist-packages/sklearn/feature_selection/_univariate_selection.py:299: RuntimeWarning: divide by zero encountered in true_divide
  corr /= X_norms
/usr/local/lib/python3.6/dist-packages/sklearn/feature_selection/_univariate_selection.py:304: RuntimeWarning: invalid value encountered in true_divide
  F = corr ** 2 / (1 - corr ** 2) * degrees_of_freedom
/usr/local/lib/python3.6/dist-packages/scipy/stats/_distn_infrastructure.py:903: RuntimeWarning: invalid value encountered in greater
  r

Test Mean Absolute Error: $157,741 

38 features
Test Mean Absolute Error: $158,747 

39 features
Test Mean Absolute Error: $158,053 

40 features
Test Mean Absolute Error: $158,102 

41 features
Test Mean Absolute Error: $158,117 

42 features
Test Mean Absolute Error: $158,456 

43 features
Test Mean Absolute Error: $158,413 

44 features
Test Mean Absolute Error: $158,408 

45 features
Test Mean Absolute Error: $158,406 

46 features
Test Mean Absolute Error: $158,407 

47 features
Test Mean Absolute Error: $158,409 

48 features
Test Mean Absolute Error: $158,410 



/usr/local/lib/python3.6/dist-packages/sklearn/feature_selection/_univariate_selection.py:299: RuntimeWarning: divide by zero encountered in true_divide
  corr /= X_norms
/usr/local/lib/python3.6/dist-packages/sklearn/feature_selection/_univariate_selection.py:304: RuntimeWarning: invalid value encountered in true_divide
  F = corr ** 2 / (1 - corr ** 2) * degrees_of_freedom
/usr/local/lib/python3.6/dist-packages/scipy/stats/_distn_infrastructure.py:903: RuntimeWarning: invalid value encountered in greater
  return (a < x) & (x < b)
/usr/local/lib/python3.6/dist-packages/scipy/stats/_distn_infrastructure.py:903: RuntimeWarning: invalid value encountered in less
  return (a < x) & (x < b)
/usr/local/lib/python3.6/dist-packages/scipy/stats/_distn_infrastructure.py:1912: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= _a)
/usr/local/lib/python3.6/dist-packages/sklearn/feature_selection/_univariate_selection.py:299: RuntimeWarning: divide by zero encountered

In [0]:
# Looks like 28 features is the best # to use when fitting a Linear Regression Model

In [244]:
# TODO: How many features should be selected?

# You can try a range of values for k,
# then choose the model with the best score.
# If multiple models "tie" for the best score,
# choose the simplest model.
# You decide what counts as a tie!

from sklearn.linear_model import Ridge
from sklearn.metrics import mean_absolute_error

for k in range(1, len(X_train.columns)+1):
    print(f'{k} features')
    
    selector = SelectKBest(score_func=f_regression, k=k)
    X_train_selected = selector.fit_transform(X_train, y_train)
    X_test_selected = selector.transform(X_test)

    model = Ridge(normalize=True)
    model.fit(X_train_selected, y_train)
    y_pred = model.predict(X_test_selected)
    mae = mean_absolute_error(y_test, y_pred)
    print(f'Test Mean Absolute Error: ${mae:,.0f} \n')

1 features
Test Mean Absolute Error: $192,592 

2 features
Test Mean Absolute Error: $191,830 

3 features
Test Mean Absolute Error: $191,575 

4 features
Test Mean Absolute Error: $190,945 

5 features


/usr/local/lib/python3.6/dist-packages/sklearn/feature_selection/_univariate_selection.py:299: RuntimeWarning: divide by zero encountered in true_divide
  corr /= X_norms
/usr/local/lib/python3.6/dist-packages/sklearn/feature_selection/_univariate_selection.py:304: RuntimeWarning: invalid value encountered in true_divide
  F = corr ** 2 / (1 - corr ** 2) * degrees_of_freedom
/usr/local/lib/python3.6/dist-packages/scipy/stats/_distn_infrastructure.py:903: RuntimeWarning: invalid value encountered in greater
  return (a < x) & (x < b)
/usr/local/lib/python3.6/dist-packages/scipy/stats/_distn_infrastructure.py:903: RuntimeWarning: invalid value encountered in less
  return (a < x) & (x < b)
/usr/local/lib/python3.6/dist-packages/scipy/stats/_distn_infrastructure.py:1912: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= _a)
/usr/local/lib/python3.6/dist-packages/sklearn/feature_selection/_univariate_selection.py:299: RuntimeWarning: divide by zero encountered

Test Mean Absolute Error: $189,503 

6 features
Test Mean Absolute Error: $186,490 

7 features
Test Mean Absolute Error: $186,795 

8 features
Test Mean Absolute Error: $186,231 

9 features
Test Mean Absolute Error: $184,869 

10 features
Test Mean Absolute Error: $183,946 

11 features
Test Mean Absolute Error: $183,396 

12 features
Test Mean Absolute Error: $176,560 

13 features
Test Mean Absolute Error: $176,817 

14 features
Test Mean Absolute Error: $176,239 

15 features
Test Mean Absolute Error: $176,240 

16 features
Test Mean Absolute Error: $176,275 

17 features
Test Mean Absolute Error: $176,294 

18 features
Test Mean Absolute Error: $176,237 

19 features
Test Mean Absolute Error: $176,218 

20 features
Test Mean Absolute Error: $174,916 

21 features


/usr/local/lib/python3.6/dist-packages/scipy/stats/_distn_infrastructure.py:1912: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= _a)
/usr/local/lib/python3.6/dist-packages/sklearn/feature_selection/_univariate_selection.py:299: RuntimeWarning: divide by zero encountered in true_divide
  corr /= X_norms
/usr/local/lib/python3.6/dist-packages/sklearn/feature_selection/_univariate_selection.py:304: RuntimeWarning: invalid value encountered in true_divide
  F = corr ** 2 / (1 - corr ** 2) * degrees_of_freedom
/usr/local/lib/python3.6/dist-packages/scipy/stats/_distn_infrastructure.py:903: RuntimeWarning: invalid value encountered in greater
  return (a < x) & (x < b)
/usr/local/lib/python3.6/dist-packages/scipy/stats/_distn_infrastructure.py:903: RuntimeWarning: invalid value encountered in less
  return (a < x) & (x < b)
/usr/local/lib/python3.6/dist-packages/scipy/stats/_distn_infrastructure.py:1912: RuntimeWarning: invalid value encountered in less_equal

Test Mean Absolute Error: $174,345 

22 features
Test Mean Absolute Error: $174,384 

23 features
Test Mean Absolute Error: $174,398 

24 features
Test Mean Absolute Error: $174,330 

25 features
Test Mean Absolute Error: $174,313 

26 features
Test Mean Absolute Error: $174,304 

27 features
Test Mean Absolute Error: $174,298 

28 features
Test Mean Absolute Error: $174,297 

29 features
Test Mean Absolute Error: $174,649 

30 features
Test Mean Absolute Error: $174,800 

31 features
Test Mean Absolute Error: $174,851 

32 features
Test Mean Absolute Error: $173,334 

33 features
Test Mean Absolute Error: $173,423 

34 features
Test Mean Absolute Error: $173,463 

35 features
Test Mean Absolute Error: $173,220 

36 features
Test Mean Absolute Error: $172,680 

37 features
Test Mean Absolute Error: $172,595 

38 features
Test Mean Absolute Error: $172,643 

39 features
Test Mean Absolute Error: $172,491 

40 features
Test Mean Absolute Error: $172,418 

41 features


/usr/local/lib/python3.6/dist-packages/sklearn/feature_selection/_univariate_selection.py:299: RuntimeWarning: divide by zero encountered in true_divide
  corr /= X_norms
/usr/local/lib/python3.6/dist-packages/sklearn/feature_selection/_univariate_selection.py:304: RuntimeWarning: invalid value encountered in true_divide
  F = corr ** 2 / (1 - corr ** 2) * degrees_of_freedom
/usr/local/lib/python3.6/dist-packages/scipy/stats/_distn_infrastructure.py:903: RuntimeWarning: invalid value encountered in greater
  return (a < x) & (x < b)
/usr/local/lib/python3.6/dist-packages/scipy/stats/_distn_infrastructure.py:903: RuntimeWarning: invalid value encountered in less
  return (a < x) & (x < b)
/usr/local/lib/python3.6/dist-packages/scipy/stats/_distn_infrastructure.py:1912: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= _a)
/usr/local/lib/python3.6/dist-packages/sklearn/feature_selection/_univariate_selection.py:299: RuntimeWarning: divide by zero encountered

Test Mean Absolute Error: $172,407 

42 features
Test Mean Absolute Error: $172,330 

43 features
Test Mean Absolute Error: $172,151 

44 features
Test Mean Absolute Error: $172,081 

45 features
Test Mean Absolute Error: $172,077 

46 features
Test Mean Absolute Error: $172,238 

47 features
Test Mean Absolute Error: $172,238 

48 features
Test Mean Absolute Error: $172,238 



/usr/local/lib/python3.6/dist-packages/sklearn/feature_selection/_univariate_selection.py:299: RuntimeWarning: divide by zero encountered in true_divide
  corr /= X_norms
/usr/local/lib/python3.6/dist-packages/sklearn/feature_selection/_univariate_selection.py:304: RuntimeWarning: invalid value encountered in true_divide
  F = corr ** 2 / (1 - corr ** 2) * degrees_of_freedom
/usr/local/lib/python3.6/dist-packages/scipy/stats/_distn_infrastructure.py:903: RuntimeWarning: invalid value encountered in greater
  return (a < x) & (x < b)
/usr/local/lib/python3.6/dist-packages/scipy/stats/_distn_infrastructure.py:903: RuntimeWarning: invalid value encountered in less
  return (a < x) & (x < b)
/usr/local/lib/python3.6/dist-packages/scipy/stats/_distn_infrastructure.py:1912: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= _a)
/usr/local/lib/python3.6/dist-packages/sklearn/feature_selection/_univariate_selection.py:299: RuntimeWarning: divide by zero encountered

In [0]:
# Looks like 45 features is the best # to use when fitting a Ridge Regression Model

In [249]:
# Run logistic Regression and test for MAE
# Here I import the class, substantiate it, fit it the model, and then apply 
# the model

from sklearn.linear_model import LinearRegression

model = LinearRegression()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

LR_mae = mean_absolute_error(y_test, y_pred)
print(f'Test Mean Absolute Error from doing Linear Regression: ${LR_mae:.0f}')

Test Mean Absolute Error from doing Linear Regression: $158410
